In [4]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!unzip -q '/content/drive/MyDrive/datasetoxfordclass.zip' -d '/content/'

# Task
Fine-tune a MobileNetV2 model for image classification using the dataset located at `/content/` to classify images and summarize the fine-tuning process and its results.

## Khám phá tập dữ liệu

### Subtask:
Liệt kê nội dung của thư mục đã giải nén (`/content/`) để hiểu cấu trúc tập dữ liệu và xác định vị trí hình ảnh cũng như nhãn.


**Reasoning**:
I need to list the contents of the `/content/` directory to understand the dataset structure and identify where the images and labels are located, as per the first instruction.



In [6]:
import os

# List the contents of the /content/ directory
print('Listing contents of /content/:')
!ls -F /content/

Listing contents of /content/:
annotations/  drive/  images/  sample_data/


**Reasoning**:
The previous output showed `images/` and `annotations/` directories. To further understand the dataset structure and locate images, I will list the contents of the `images/` directory.



In [7]:
print('Listing contents of /content/images/:')
!ls -F /content/images/

Listing contents of /content/images/:
images/


**Reasoning**:
The previous output indicated a nested directory structure (`/content/images/images/`). To find the actual image files and understand their arrangement, I need to list the contents of the innermost `images/` directory.



In [8]:
print('Listing contents of /content/images/images/:')
!ls -F /content/images/images/ | head -n 10

Listing contents of /content/images/images/:
Abyssinian_100.jpg
Abyssinian_100.mat
Abyssinian_101.jpg
Abyssinian_101.mat
Abyssinian_102.jpg
Abyssinian_102.mat
Abyssinian_103.jpg
Abyssinian_104.jpg
Abyssinian_105.jpg
Abyssinian_106.jpg


**Reasoning**:
The previous steps explored the image directory. Now, to understand the dataset structure comprehensively and locate the labels as per the subtask, I need to list the contents of the `annotations/` directory, which is likely to contain label information.



In [9]:
print('Listing contents of /content/annotations/:')
!ls -F /content/annotations/ | head -n 10

Listing contents of /content/annotations/:
annotations/


**Reasoning**:
The previous output for `/content/annotations/` showed a nested `annotations/` directory. To find the actual annotation files and understand their structure, I need to list the contents of the innermost `/content/annotations/annotations/` directory.



In [10]:
print('Listing contents of /content/annotations/annotations/:')
!ls -F /content/annotations/annotations/ | head -n 10

Listing contents of /content/annotations/annotations/:
list.txt
README
test.txt
trainval.txt
trimaps/
xmls/


## Chuẩn bị dữ liệu

### Subtask:
Tạo các bộ tải dữ liệu (data loaders) cho tập huấn luyện và tập xác thực, bao gồm tải hình ảnh, thay đổi kích thước, chuẩn hóa và mã hóa nhãn.


**Reasoning**:
I need to read the `list.txt` file located in `/content/annotations/annotations/` to extract all unique class names, which is the first part of the instruction to analyze annotation files and understand the dataset's class structure. This file typically contains class definitions or a list of categories.



In [11]:
import os
import re

annotations_dir = '/content/annotations/annotations/'
list_file_path = os.path.join(annotations_dir, 'list.txt')

# Read list.txt to extract class names and store them
class_names = []
with open(list_file_path, 'r') as f:
    for line in f:
        line = line.strip()
        if line.startswith('#'): # Skip comments
            continue
        # Example line: Abyssinian 1 1 1
        # Extract the breed name, which is the class name
        parts = line.split(' ')
        if parts:
            class_name = parts[0]
            class_names.append(class_name)

# Get unique class names and sort them for consistent indexing
unique_class_names = sorted(list(set(class_names)))
num_classes = len(unique_class_names)

# Create a mapping from class name to integer label
class_to_label = {name: i for i, name in enumerate(unique_class_names)}
label_to_class = {i: name for i, name in enumerate(unique_class_names)}

print(f"Number of unique classes: {num_classes}")
print("First 5 class names:")
for i in range(min(5, num_classes)):
    print(f"  {unique_class_names[i]}")
print("Class to label mapping (first 5):")
for i, (k, v) in enumerate(class_to_label.items()):
    if i >= 5: break
    print(f"  '{k}': {v}")

Number of unique classes: 7349
First 5 class names:
  Abyssinian_1
  Abyssinian_10
  Abyssinian_100
  Abyssinian_101
  Abyssinian_102
Class to label mapping (first 5):
  'Abyssinian_1': 0
  'Abyssinian_10': 1
  'Abyssinian_100': 2
  'Abyssinian_101': 3
  'Abyssinian_102': 4


**Reasoning**:
Following the first instruction of the subtask, I have already processed `list.txt`. Now, I need to read and parse `trainval.txt` and `test.txt` to extract image filenames and their corresponding class labels. This will create the necessary data for the training, validation, and test sets.



In [12]:
import os

image_base_path = '/content/images/images/'

def parse_annotation_file(file_path, class_to_label):
    image_paths = []
    labels = []
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('#'):
                continue
            # Example line: Abyssinian_100 1 1 1
            parts = line.split(' ')
            if len(parts) >= 1:
                image_name_prefix = parts[0]
                # The image file names are typically <class_name>_<image_number>.jpg
                # The class name itself is the prefix before the last underscore, e.g., 'Abyssinian'
                # Need to find the actual class name based on the structure.
                # From previous step, list.txt entries were like 'Abyssinian_100', which might be a breed instance.
                # However, the task implies classification of broader categories (breeds).
                # Let's re-evaluate how class names are extracted from the image_name_prefix.

                # Looking at typical Oxford-IIIT Pet dataset format:
                # Image file names: <breed_name>_<image_id>.jpg (e.g., Abyssinian_1.jpg)
                # Annotation lines: <image_name> <class_id> <species> <breed_id>
                # The problem statement states 'fine-tune a MobileNetV2 model for image classification'.
                # The `list.txt` processing resulted in 7349 unique classes like 'Abyssinian_1', 'Abyssinian_10'.
                # This suggests that each image instance might be treated as a unique class, which is unusual for breed classification.
                # Let's assume for now the *actual* class name is the breed name before the last underscore.

                # Revised approach to extract class name from image_name_prefix:
                # Example: 'Abyssinian_100' -> actual breed name 'Abyssinian'
                # We need to map this 'Abyssinian' to a label.

                # Let's re-examine the structure of list.txt and common practice for Oxford-IIIT Pet.
                # list.txt has lines like: 'Abyssinian_1 1 1 1'
                # The first part 'Abyssinian_1' is the image filename prefix.
                # The actual breed name is usually the part before the last underscore, e.g., 'Abyssinian'.
                # Let's define the class names based on the *breed* rather than the instance.

                # Let's rebuild unique_class_names and class_to_label to reflect actual breeds if that was the intent.
                # However, the previous `class_to_label` created 7349 classes from `list.txt` based on `parts[0]`.
                # If the goal is to classify 37 breeds, then `num_classes` should be 37.
                # The original `list.txt` processing seems to have interpreted each distinct image prefix as a class.
                # This needs correction if the goal is breed classification.

                # Let's stick to the output of `list.txt` processing for now, which created labels for specific image prefixes.
                # The problem statement says 'classify images' and 'fine-tune a MobileNetV2 model for image classification'.
                # It does not explicitly state 'breed classification'.
                # The number of classes (7349) is very high for standard image classification datasets.
                # It's more likely that the output `Abyssinian_1`, `Abyssinian_10` are the *class labels* themselves (e.g., specific animals).
                # This interpretation would mean the dataset is about individual identification rather than breed.
                # Given the `list.txt` structure, where each entry is unique and contains an 'image name prefix' as the first part,
                # it implies `parts[0]` *is* the class name for that image.

                # Let's assume `parts[0]` from `trainval.txt` and `test.txt` directly correspond to the keys in `class_to_label`.

                image_name_prefix = parts[0]
                class_idx = class_to_label[image_name_prefix]

                # Construct the full image path
                full_image_path = os.path.join(image_base_path, image_name_prefix + '.jpg')

                image_paths.append(full_image_path)
                labels.append(class_idx)
    return image_paths, labels

# Paths to trainval.txt and test.txt
trainval_file_path = os.path.join(annotations_dir, 'trainval.txt')
test_file_path = os.path.join(annotations_dir, 'test.txt')

# Parse trainval.txt
trainval_image_paths, trainval_labels = parse_annotation_file(trainval_file_path, class_to_label)
print(f"Found {len(trainval_image_paths)} images for training/validation.")
print(f"First 5 training/validation image paths and labels:")
for i in range(min(5, len(trainval_image_paths))):
    print(f"  Path: {trainval_image_paths[i]}, Label: {trainval_labels[i]} ({label_to_class[trainval_labels[i]]})")

# Parse test.txt
test_image_paths, test_labels = parse_annotation_file(test_file_path, class_to_label)
print(f"Found {len(test_image_paths)} images for testing.")
print(f"First 5 testing image paths and labels:")
for i in range(min(5, len(test_image_paths))):
    print(f"  Path: {test_image_paths[i]}, Label: {test_labels[i]} ({label_to_class[test_labels[i]]})")


# Now, split the trainval set into actual training and validation sets
from sklearn.model_selection import train_test_split

train_image_paths, val_image_paths, train_labels, val_labels = train_test_split(
    trainval_image_paths, trainval_labels, test_size=0.2, random_state=42, stratify=trainval_labels
)

print(f"\nAfter splitting trainval set:")
print(f"  Training images: {len(train_image_paths)}")
print(f"  Validation images: {len(val_image_paths)}")
print(f"  Test images: {len(test_image_paths)}")

Found 3680 images for training/validation.
First 5 training/validation image paths and labels:
  Path: /content/images/images/Abyssinian_100.jpg, Label: 2 (Abyssinian_100)
  Path: /content/images/images/Abyssinian_101.jpg, Label: 3 (Abyssinian_101)
  Path: /content/images/images/Abyssinian_102.jpg, Label: 4 (Abyssinian_102)
  Path: /content/images/images/Abyssinian_103.jpg, Label: 5 (Abyssinian_103)
  Path: /content/images/images/Abyssinian_104.jpg, Label: 6 (Abyssinian_104)
Found 3669 images for testing.
First 5 testing image paths and labels:
  Path: /content/images/images/Abyssinian_201.jpg, Label: 102 (Abyssinian_201)
  Path: /content/images/images/Abyssinian_202.jpg, Label: 103 (Abyssinian_202)
  Path: /content/images/images/Abyssinian_204.jpg, Label: 104 (Abyssinian_204)
  Path: /content/images/images/Abyssinian_205.jpg, Label: 105 (Abyssinian_205)
  Path: /content/images/images/Abyssinian_206.jpg, Label: 106 (Abyssinian_206)


ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

**Reasoning**:
The previous `train_test_split` failed because many 'classes' had only one sample, which is incompatible with stratified splitting. This was due to incorrectly identifying each unique image identifier (e.g., 'Abyssinian_100') as a separate class. The Oxford-IIIT Pet dataset typically classifies by breed. I need to modify the class extraction logic to identify actual breed names (e.g., 'Abyssinian') as classes, which will result in a reasonable number of classes (around 37) and allow for successful stratified splitting.



In [13]:
import os
import re
from sklearn.model_selection import train_test_split

annotations_dir = '/content/annotations/annotations/'
list_file_path = os.path.join(annotations_dir, 'list.txt')

# Step 1: Re-evaluate class extraction to get actual breed names
# The original list.txt interpretation created too many classes (individual image identifiers)
# For Oxford-IIIT Pet, classes are typically breeds.
# Let's derive breed names from the entries in list.txt (or image filenames).

# Function to extract breed name from a string like 'Abyssinian_100'
def extract_breed_name(image_name_prefix):
    # The breed name is everything before the last underscore
    return '_'.join(image_name_prefix.split('_')[:-1])

# Read list.txt to extract actual breed names
breed_names = []
with open(list_file_path, 'r') as f:
    for line in f:
        line = line.strip()
        if line.startswith('#'): # Skip comments
            continue
        parts = line.split(' ')
        if parts:
            image_name_prefix = parts[0]
            breed_name = extract_breed_name(image_name_prefix)
            if breed_name:
                breed_names.append(breed_name)

# Get unique breed names and sort them for consistent indexing
unique_breed_names = sorted(list(set(breed_names)))
num_classes = len(unique_breed_names)

# Create a mapping from breed name to integer label
class_to_label = {name: i for i, name in enumerate(unique_breed_names)}
label_to_class = {i: name for i, name in enumerate(unique_breed_names)}

print(f"Number of actual breed classes: {num_classes}")
print("First 5 breed names:")
for i in range(min(5, num_classes)):
    print(f"  {unique_breed_names[i]}")
print("Class to label mapping (first 5):")
for i, (k, v) in enumerate(class_to_label.items()):
    if i >= 5: break
    print(f"  '{k}': {v}")

image_base_path = '/content/images/images/'

def parse_annotation_file(file_path, class_to_label):
    image_paths = []
    labels = []
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('#'):
                continue
            parts = line.split(' ')
            if len(parts) >= 1:
                image_name_prefix = parts[0]
                breed_name = extract_breed_name(image_name_prefix)

                if breed_name in class_to_label:
                    class_idx = class_to_label[breed_name]
                    full_image_path = os.path.join(image_base_path, image_name_prefix + '.jpg')

                    # Check if the image file actually exists
                    if os.path.exists(full_image_path):
                        image_paths.append(full_image_path)
                        labels.append(class_idx)
                    # else: # Optional: print a warning if image not found
                    #     print(f"Warning: Image file not found: {full_image_path}")
                # else: # Optional: print a warning if breed name not in our defined classes
                #     print(f"Warning: Breed name '{breed_name}' not found in class_to_label for image '{image_name_prefix}'")
    return image_paths, labels

# Paths to trainval.txt and test.txt
trainval_file_path = os.path.join(annotations_dir, 'trainval.txt')
test_file_path = os.path.join(annotations_dir, 'test.txt')

# Parse trainval.txt
trainval_image_paths, trainval_labels = parse_annotation_file(trainval_file_path, class_to_label)
print(f"\nFound {len(trainval_image_paths)} images for training/validation (based on actual breeds).")
print(f"First 5 training/validation image paths and labels:")
for i in range(min(5, len(trainval_image_paths))):
    print(f"  Path: {trainval_image_paths[i]}, Label: {trainval_labels[i]} ({label_to_class[trainval_labels[i]]})")

# Parse test.txt
test_image_paths, test_labels = parse_annotation_file(test_file_path, class_to_label)
print(f"\nFound {len(test_image_paths)} images for testing (based on actual breeds).")
print(f"First 5 testing image paths and labels:")
for i in range(min(5, len(test_image_paths))):
    print(f"  Path: {test_image_paths[i]}, Label: {test_labels[i]} ({label_to_class[test_labels[i]]})")


# Now, split the trainval set into actual training and validation sets
# With fewer, correctly defined classes, stratification should now work.
train_image_paths, val_image_paths, train_labels, val_labels = train_test_split(
    trainval_image_paths, trainval_labels, test_size=0.2, random_state=42, stratify=trainval_labels
)

print(f"\nAfter splitting trainval set:")
print(f"  Training images: {len(train_image_paths)}")
print(f"  Validation images: {len(val_image_paths)}")
print(f"  Test images: {len(test_image_paths)}")


Number of actual breed classes: 37
First 5 breed names:
  Abyssinian
  Bengal
  Birman
  Bombay
  British_Shorthair
Class to label mapping (first 5):
  'Abyssinian': 0
  'Bengal': 1
  'Birman': 2
  'Bombay': 3
  'British_Shorthair': 4

Found 3680 images for training/validation (based on actual breeds).
First 5 training/validation image paths and labels:
  Path: /content/images/images/Abyssinian_100.jpg, Label: 0 (Abyssinian)
  Path: /content/images/images/Abyssinian_101.jpg, Label: 0 (Abyssinian)
  Path: /content/images/images/Abyssinian_102.jpg, Label: 0 (Abyssinian)
  Path: /content/images/images/Abyssinian_103.jpg, Label: 0 (Abyssinian)
  Path: /content/images/images/Abyssinian_104.jpg, Label: 0 (Abyssinian)

Found 3669 images for testing (based on actual breeds).
First 5 testing image paths and labels:
  Path: /content/images/images/Abyssinian_201.jpg, Label: 0 (Abyssinian)
  Path: /content/images/images/Abyssinian_202.jpg, Label: 0 (Abyssinian)
  Path: /content/images/images/Abyss

**Reasoning**:
The previous steps successfully extracted image paths and labels for training, validation, and testing. The next step is to define an image preprocessing function and create TensorFlow dataset objects as per instructions 4, 5, and 6 of the subtask. This involves reading the image, decoding it, resizing it to the MobileNetV2 input size (224x224), and normalizing pixel values. Then, `tf.data.Dataset` objects will be created, mapping the preprocessing function and configuring them for optimal performance (shuffling, batching, prefetching).



In [14]:
import tensorflow as tf

IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

# Define the preprocessing function
def preprocess_image(image_path, label):
    # Read the image file
    image = tf.io.read_file(image_path)
    # Decode the image to a tensor
    image = tf.image.decode_jpeg(image, channels=3)
    # Resize the image to the desired size for MobileNetV2
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
    # Normalize pixel values to the range [-1, 1]
    # MobileNetV2 expects input pixel values to be in the range [-1, 1]
    image = (image / 127.5) - 1
    return image, label

# Create tf.data.Dataset objects for training, validation, and test sets
def create_dataset(image_paths, labels, batch_size=BATCH_SIZE, shuffle=True, augment=False):
    # Create a dataset from image paths and labels
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))

    # Apply preprocessing
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)

    # Configure for performance
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(image_paths))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

# Create datasets
train_dataset = create_dataset(train_image_paths, train_labels, shuffle=True)
val_dataset = create_dataset(val_image_paths, val_labels, shuffle=False)
test_dataset = create_dataset(test_image_paths, test_labels, shuffle=False)

print(f"Train dataset created with batch size {BATCH_SIZE}")
print(f"Validation dataset created with batch size {BATCH_SIZE}")
print(f"Test dataset created with batch size {BATCH_SIZE}")

# Verify a batch from the training dataset
for images, labels in train_dataset.take(1):
    print(f"Sample batch shape - Images: {images.shape}, Labels: {labels.shape}")
    break


Train dataset created with batch size 32
Validation dataset created with batch size 32
Test dataset created with batch size 32
Sample batch shape - Images: (32, 224, 224, 3), Labels: (32,)


## Tải mô hình MobileNetV2

### Subtask:
Tải mô hình MobileNetV2 đã được huấn luyện trước (pre-trained) từ TensorFlow/Keras, loại bỏ lớp phân loại trên cùng.


**Reasoning**:
As per the subtask instruction, I need to load the pre-trained MobileNetV2 model from TensorFlow/Keras, ensuring that the top classification layer is excluded and ImageNet weights are used. I will use the previously defined IMG_HEIGHT and IMG_WIDTH for the input shape.



In [15]:
import tensorflow as tf

# Import MobileNetV2
from tensorflow.keras.applications import MobileNetV2

# Instantiate the MobileNetV2 model with pre-trained ImageNet weights,
# excluding the top classification layer.
# The input_shape is derived from the previously defined IMG_HEIGHT and IMG_WIDTH.
base_model = MobileNetV2(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
                         include_top=False,
                         weights='imagenet')

print("MobileNetV2 base model loaded successfully with ImageNet weights and without the top layer.")
print(f"Base model output shape: {base_model.output_shape}")

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
MobileNetV2 base model loaded successfully with ImageNet weights and without the top layer.
Base model output shape: (None, 7, 7, 1280)


## Sửa đổi mô hình để tinh chỉnh

### Subtask:
Thêm các lớp phân loại mới phù hợp với số lượng lớp duy nhất trong tập dữ liệu của bạn.


**Reasoning**:
As per the instructions, I need to add new classification layers on top of the pre-trained MobileNetV2 base model. This involves freezing the base model, defining a new sequential head with GlobalAveragePooling2D, a dense layer, and a final dense classification layer with softmax activation, and then combining them into a final model. Finally, I will print the model summary.



In [16]:
import tensorflow as tf

# 1. Freeze the base model
base_model.trainable = False

# 2. Create a new classification head
# The number of classes was determined in the previous step (num_classes)

head = tf.keras.Sequential([
    tf.keras.layers.GlobalAveragePooling2D(), # Converts the 7x7x1280 feature map into a 1280-element vector
    tf.keras.layers.Dense(128, activation='relu'), # A dense hidden layer
    tf.keras.layers.Dense(num_classes, activation='softmax') # The final classification layer
])

# 6. Create the complete model by connecting the base_model and the head
model = tf.keras.Sequential([
    base_model,
    head
])

# 7. Print the model summary to verify the new architecture
print("Model architecture after adding classification head:")
model.summary()

Model architecture after adding classification head:


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 37)             │       168,741 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,426,725 (9.26 MB)

 Trainable params: 168,741 (659.14 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

## Biên dịch mô hình

### Subtask:
Cấu hình mô hình với trình tối ưu hóa (optimizer), hàm mất mát (loss function) và các chỉ số (metrics) phù hợp để huấn luyện.


**Reasoning**:
As per the instructions, I need to configure the model by selecting an optimizer (Adam), a loss function (`SparseCategoricalCrossentropy` for multi-class classification with integer labels and softmax output), and metrics (accuracy) and then compile the model using these.



In [17]:
import tensorflow as tf

# 1. Choose an optimizer
optimizer = tf.keras.optimizers.Adam()

# 2. Choose a loss function suitable for multi-class classification
# Since our labels are integer-encoded (0, 1, 2, ...) and the output layer uses 'softmax' activation,
# SparseCategoricalCrossentropy is the correct choice, with from_logits=False.
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

# 3. Choose metrics to monitor model performance
metrics = ['accuracy']

# 4. Compile the model
model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=metrics)

print("Model compiled successfully with Adam optimizer, SparseCategoricalCrossentropy loss, and accuracy metric.")

Model compiled successfully with Adam optimizer, SparseCategoricalCrossentropy loss, and accuracy metric.


## Tinh chỉnh mô hình

### Subtask:
Tiến hành huấn luyện mô hình MobileNetV2 đã sửa đổi trên tập dữ liệu đã chuẩn bị.


**Reasoning**:
As per the instructions, I need to train the compiled model using the prepared training and validation datasets. I will specify a suitable number of epochs and store the training history.



In [1]:
EPOCHS = 10 # Define the number of training epochs

# Train the model
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset
)

print(f"Model training completed for {EPOCHS} epochs.")

NameError: name 'model' is not defined

# Task
```python
import tensorflow as tf

# Freeze the base model if it's not already frozen (ensures transfer learning)
base_model.trainable = False

# Create a new classification head (re-using num_classes from previous steps)
head = tf.keras.Sequential([
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Create the complete model by connecting the base_model and the head
model = tf.keras.Sequential([
    base_model,
    head
])

print("Model architecture after adding classification head:")
model.summary()

# Compile the model
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
metrics = ['accuracy']

model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=metrics)

print("\nModel compiled successfully with Adam optimizer, SparseCategoricalCrossentropy loss, and accuracy metric.")

# Define the number of training epochs
EPOCHS = 10

# Train the model
print(f"\nStarting model training for {EPOCHS} epochs...")
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset
)

print(f"\nModel training completed for {EPOCHS} epochs.")
```

## Thiết lập GPU

### Subtask:
Kiểm tra và cấu hình môi trường để sử dụng GPU nhằm tăng tốc quá trình huấn luyện.


**Reasoning**:
The previous code block resulted in a `NameError` because the `model` object was not defined when the cell was executed. This indicates a state loss or an issue with the execution order. Before proceeding with training, I need to ensure the `model` object is defined.



In [2]:
import tensorflow as tf

# Import MobileNetV2
from tensorflow.keras.applications import MobileNetV2

# Re-define IMG_HEIGHT, IMG_WIDTH, num_classes if they are not in the current kernel state
# Assuming these were defined in previous successful steps, but for robustness, include them if necessary.
# For example, if you ran the notebook from scratch up to the error, these should be defined.
# If not, you might need to re-run the cells that define them.
# Let's assume for now that num_classes, IMG_HEIGHT, IMG_WIDTH are available from previous successful runs.

# Placeholder values if not available (ideally should come from previous executed cells)
# num_classes = 37 # Should be derived from actual class parsing
# IMG_HEIGHT = 224
# IMG_WIDTH = 224

# Instantiate the MobileNetV2 model with pre-trained ImageNet weights,
# excluding the top classification layer.
base_model = MobileNetV2(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
                         include_top=False,
                         weights='imagenet')

# Freeze the base model
base_model.trainable = False

# Create a new classification head
head = tf.keras.Sequential([
    tf.keras.layers.GlobalAveragePooling2D(), # Converts the 7x7x1280 feature map into a 1280-element vector
    tf.keras.layers.Dense(128, activation='relu'), # A dense hidden layer
    tf.keras.layers.Dense(num_classes, activation='softmax') # The final classification layer
])

# Create the complete model by connecting the base_model and the head
model = tf.keras.Sequential([
    base_model,
    head
])

# Compile the model (assuming optimizer, loss_fn, metrics are available)
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
metrics = ['accuracy']

model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=metrics)

print("Model rebuilt and compiled successfully.")
print(f"Number of classes used for the final layer: {num_classes}")

NameError: name 'IMG_HEIGHT' is not defined

**Reasoning**:
The previous code block failed because `IMG_HEIGHT`, `IMG_WIDTH`, and `num_classes` were not defined in the current execution scope. I need to re-declare these variables with their established values before initializing and compiling the model.



In [3]:
import tensorflow as tf

# Import MobileNetV2
from tensorflow.keras.applications import MobileNetV2

# Re-define IMG_HEIGHT, IMG_WIDTH, num_classes from previous successful steps
# These values were determined in cells 3907940b and f0e9cee2 respectively.
IMG_HEIGHT = 224
IMG_WIDTH = 224
num_classes = 37 # This was determined after correctly parsing breed names

# Instantiate the MobileNetV2 model with pre-trained ImageNet weights,
# excluding the top classification layer.
base_model = MobileNetV2(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
                         include_top=False,
                         weights='imagenet')

# Freeze the base model
base_model.trainable = False

# Create a new classification head
head = tf.keras.Sequential([
    tf.keras.layers.GlobalAveragePooling2D(), # Converts the 7x7x1280 feature map into a 1280-element vector
    tf.keras.layers.Dense(128, activation='relu'), # A dense hidden layer
    tf.keras.layers.Dense(num_classes, activation='softmax') # The final classification layer
])

# Create the complete model by connecting the base_model and the head
model = tf.keras.Sequential([
    base_model,
    head
])

# Compile the model (assuming optimizer, loss_fn, metrics are available)
optimizer = tf.keras.optimizers.Adam()
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
metrics = ['accuracy']

model.compile(optimizer=optimizer,
              loss=loss_fn,
              metrics=metrics)

print("Model rebuilt and compiled successfully.")
print(f"Number of classes used for the final layer: {num_classes}")

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Model rebuilt and compiled successfully.
Number of classes used for the final layer: 37


**Reasoning**:
As per the subtask instructions, I need to check for GPU availability. I will use `tf.config.list_physical_devices('GPU')` to list the physical GPU devices that TensorFlow can utilize.



In [4]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"Found {len(gpus)} GPU(s):")
    for gpu in gpus:
        print(f"  {gpu}")
    print("GPU is available and configured for TensorFlow.")
else:
    print("No GPU devices found. TensorFlow will run on CPU.")

# Optional: Run nvidia-smi for more detailed GPU information if a GPU is detected
if gpus:
    print("\nRunning !nvidia-smi for detailed GPU information:")
    !nvidia-smi

Found 1 GPU(s):
  PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
GPU is available and configured for TensorFlow.

Running !nvidia-smi for detailed GPU information:
Tue Feb  3 03:58:10 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P0             27W /   70W |     13

## Gắn Google Drive và giải nén dữ liệu

### Subtask:
Gắn Google Drive để truy cập tệp zip datasetoxfordclass.zip và giải nén nó vào thư mục /content/.


## Tinh chỉnh mô hình

### Subtask:
Tiến hành huấn luyện mô hình MobileNetV2 đã sửa đổi trên tập dữ liệu đã chuẩn bị.


**Reasoning**:
The `model` object has been successfully compiled and `IMG_HEIGHT`, `IMG_WIDTH`, and `num_classes` are defined. The `train_dataset` and `val_dataset` are also ready. I will now proceed to train the model as per the instructions, defining `EPOCHS` and calling `model.fit()` to store the training history.



In [5]:
EPOCHS = 10 # Define the number of training epochs

# Train the model
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset
)

print(f"Model training completed for {EPOCHS} epochs.")

NameError: name 'train_dataset' is not defined

**Reasoning**:
The previous code block failed because 'train_dataset' was not defined. This indicates that the data parsing and dataset creation steps were not executed or their variables were lost. I need to re-run the code for parsing annotation files, splitting data into training and validation sets, and creating TensorFlow dataset objects before initiating model training.



In [6]:
import os
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split

# --- Re-execute Data Parsing and Splitting (from f0e9cee2) ---
annotations_dir = '/content/annotations/annotations/'
list_file_path = os.path.join(annotations_dir, 'list.txt')

def extract_breed_name(image_name_prefix):
    return '_'.join(image_name_prefix.split('_')[:-1])

breed_names = []
with open(list_file_path, 'r') as f:
    for line in f:
        line = line.strip()
        if line.startswith('#'):
            continue
        parts = line.split(' ')
        if parts:
            image_name_prefix = parts[0]
            breed_name = extract_breed_name(image_name_prefix)
            if breed_name:
                breed_names.append(breed_name)

unique_breed_names = sorted(list(set(breed_names)))
num_classes = len(unique_breed_names)

class_to_label = {name: i for i, name in enumerate(unique_breed_names)}
label_to_class = {i: name for i, name in enumerate(unique_breed_names)}

image_base_path = '/content/images/images/'

def parse_annotation_file(file_path, class_to_label):
    image_paths = []
    labels = []
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('#'):
                continue
            parts = line.split(' ')
            if len(parts) >= 1:
                image_name_prefix = parts[0]
                breed_name = extract_breed_name(image_name_prefix)

                if breed_name in class_to_label:
                    class_idx = class_to_label[breed_name]
                    full_image_path = os.path.join(image_base_path, image_name_prefix + '.jpg')

                    if os.path.exists(full_image_path):
                        image_paths.append(full_image_path)
                        labels.append(class_idx)
    return image_paths, labels

trainval_file_path = os.path.join(annotations_dir, 'trainval.txt')
test_file_path = os.path.join(annotations_dir, 'test.txt')

trainval_image_paths, trainval_labels = parse_annotation_file(trainval_file_path, class_to_label)
test_image_paths, test_labels = parse_annotation_file(test_file_path, class_to_label)

train_image_paths, val_image_paths, train_labels, val_labels = train_test_split(
    trainval_image_paths, trainval_labels, test_size=0.2, random_state=42, stratify=trainval_labels
)

# --- Re-execute Dataset Creation (from 3907940b) ---
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

def preprocess_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
    image = (image / 127.5) - 1
    return image, label

def create_dataset(image_paths, labels, batch_size=BATCH_SIZE, shuffle=True, augment=False):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(image_paths))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

train_dataset = create_dataset(train_image_paths, train_labels, shuffle=True)
val_dataset = create_dataset(val_image_paths, val_labels, shuffle=False)
test_dataset = create_dataset(test_image_paths, test_labels, shuffle=False)

print("Datasets (train_dataset, val_dataset, test_dataset) have been successfully re-created.")

# --- Model Training ---
EPOCHS = 10 # Define the number of training epochs

# Train the model
print(f"\nStarting model training for {EPOCHS} epochs...")
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset
)

print(f"\nModel training completed for {EPOCHS} epochs.")

FileNotFoundError: [Errno 2] No such file or directory: '/content/annotations/annotations/list.txt'

**Reasoning**:
The previous code block failed because the necessary dataset files, specifically `list.txt`, were not found. This indicates that the dataset was not correctly unzipped or the unzipped files were not available in the expected location. I will re-run the `unzip` command to ensure the dataset is extracted, and then proceed with re-creating the datasets (train_dataset, val_dataset, test_dataset) and finally, start the model training.



In [7]:
!unzip -q '/content/drive/MyDrive/datasetoxfordclass.zip' -d '/content/'

import os
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split

# --- Re-execute Data Parsing and Splitting (from f0e9cee2) ---
annotations_dir = '/content/annotations/annotations/'
list_file_path = os.path.join(annotations_dir, 'list.txt')

def extract_breed_name(image_name_prefix):
    return '_'.join(image_name_prefix.split('_')[:-1])

breed_names = []
with open(list_file_path, 'r') as f:
    for line in f:
        line = line.strip()
        if line.startswith('#'):
            continue
        parts = line.split(' ')
        if parts:
            image_name_prefix = parts[0]
            breed_name = extract_breed_name(image_name_prefix)
            if breed_name:
                breed_names.append(breed_name)

unique_breed_names = sorted(list(set(breed_names)))
num_classes = len(unique_breed_names)

class_to_label = {name: i for i, name in enumerate(unique_breed_names)}
label_to_class = {i: name for i, name in enumerate(unique_breed_names)}

image_base_path = '/content/images/images/'

def parse_annotation_file(file_path, class_to_label):
    image_paths = []
    labels = []
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('#'):
                continue
            parts = line.split(' ')
            if len(parts) >= 1:
                image_name_prefix = parts[0]
                breed_name = extract_breed_name(image_name_prefix)

                if breed_name in class_to_label:
                    class_idx = class_to_label[breed_name]
                    full_image_path = os.path.join(image_base_path, image_name_prefix + '.jpg')

                    if os.path.exists(full_image_path):
                        image_paths.append(full_image_path)
                        labels.append(class_idx)
    return image_paths, labels

trainval_file_path = os.path.join(annotations_dir, 'trainval.txt')
test_file_path = os.path.join(annotations_dir, 'test.txt')

trainval_image_paths, trainval_labels = parse_annotation_file(trainval_file_path, class_to_label)
test_image_paths, test_labels = parse_annotation_file(test_file_path, class_to_label)

train_image_paths, val_image_paths, train_labels, val_labels = train_test_split(
    trainval_image_paths, trainval_labels, test_size=0.2, random_state=42, stratify=trainval_labels
)

# --- Re-execute Dataset Creation (from 3907940b) ---
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

def preprocess_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
    image = (image / 127.5) - 1
    return image, label

def create_dataset(image_paths, labels, batch_size=BATCH_SIZE, shuffle=True, augment=False):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(image_paths))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

train_dataset = create_dataset(train_image_paths, train_labels, shuffle=True)
val_dataset = create_dataset(val_image_paths, val_labels, shuffle=False)
test_dataset = create_dataset(test_image_paths, test_labels, shuffle=False)

print("Datasets (train_dataset, val_dataset, test_dataset) have been successfully re-created.")

# --- Model Training ---
EPOCHS = 10 # Define the number of training epochs

# Train the model
print(f"\nStarting model training for {EPOCHS} epochs...")
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset
)

print(f"\nModel training completed for {EPOCHS} epochs.")

unzip:  cannot find or open /content/drive/MyDrive/datasetoxfordclass.zip, /content/drive/MyDrive/datasetoxfordclass.zip.zip or /content/drive/MyDrive/datasetoxfordclass.zip.ZIP.


FileNotFoundError: [Errno 2] No such file or directory: '/content/annotations/annotations/list.txt'

**Reasoning**:
The previous code block failed because the `unzip` command could not find the dataset file, leading to subsequent `FileNotFoundError` for `list.txt`. I will re-add `drive.mount` with `force_remount=True` for robustness, then re-attempt the `unzip` command. Following this, I will re-declare necessary variables and re-execute the data parsing, dataset creation, and model training steps, assuming the `model` object was successfully compiled in an earlier step and is still in scope.



In [8]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!unzip -q '/content/drive/MyDrive/datasetoxfordclass.zip' -d '/content/'

import os
import re
import tensorflow as tf
from sklearn.model_selection import train_test_split

# --- Re-define crucial variables from previous successful steps ---
# These values were determined in cells 3907940b and f0e9cee2 respectively.
IMG_HEIGHT = 224
IMG_WIDTH = 224
num_classes = 37 # This was determined after correctly parsing breed names
BATCH_SIZE = 32 # Ensure BATCH_SIZE is also defined

# --- Re-execute Data Parsing and Splitting (from f0e9cee2) ---
annotations_dir = '/content/annotations/annotations/'
list_file_path = os.path.join(annotations_dir, 'list.txt')

def extract_breed_name(image_name_prefix):
    return '_'.join(image_name_prefix.split('_')[:-1])

breed_names = []
with open(list_file_path, 'r') as f:
    for line in f:
        line = line.strip()
        if line.startswith('#'):
            continue
        parts = line.split(' ')
        if parts:
            image_name_prefix = parts[0]
            breed_name = extract_breed_name(image_name_prefix)
            if breed_name:
                breed_names.append(breed_name)

unique_breed_names = sorted(list(set(breed_names)))
num_classes = len(unique_breed_names) # Ensure num_classes is consistent

class_to_label = {name: i for i, name in enumerate(unique_breed_names)}
label_to_class = {i: name for i, name in enumerate(unique_breed_names)}

image_base_path = '/content/images/images/'

def parse_annotation_file(file_path, class_to_label):
    image_paths = []
    labels = []
    with open(file_path, 'r') as f:
        for line in f:
            line = line.strip()
            if line.startswith('#'):
                continue
            parts = line.split(' ')
            if len(parts) >= 1:
                image_name_prefix = parts[0]
                breed_name = extract_breed_name(image_name_prefix)

                if breed_name in class_to_label:
                    class_idx = class_to_label[breed_name]
                    full_image_path = os.path.join(image_base_path, image_name_prefix + '.jpg')

                    if os.path.exists(full_image_path):
                        image_paths.append(full_image_path)
                        labels.append(class_idx)
    return image_paths, labels

trainval_file_path = os.path.join(annotations_dir, 'trainval.txt')
test_file_path = os.path.join(annotations_dir, 'test.txt')

trainval_image_paths, trainval_labels = parse_annotation_file(trainval_file_path, class_to_label)
test_image_paths, test_labels = parse_annotation_file(test_file_path, class_to_label)

train_image_paths, val_image_paths, train_labels, val_labels = train_test_split(
    trainval_image_paths, trainval_labels, test_size=0.2, random_state=42, stratify=trainval_labels
)

# --- Re-execute Dataset Creation (from 3907940b) ---

def preprocess_image(image_path, label):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [IMG_HEIGHT, IMG_WIDTH])
    image = (image / 127.5) - 1
    return image, label

def create_dataset(image_paths, labels, batch_size=BATCH_SIZE, shuffle=True, augment=False):
    dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels))
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(image_paths))
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    return dataset

train_dataset = create_dataset(train_image_paths, train_labels, shuffle=True)
val_dataset = create_dataset(val_image_paths, val_labels, shuffle=False)
test_dataset = create_dataset(test_image_paths, test_labels, shuffle=False)

print("Datasets (train_dataset, val_dataset, test_dataset) have been successfully re-created.")

# --- Model Training ---
EPOCHS = 10 # Define the number of training epochs

# Train the model
print(f"\nStarting model training for {EPOCHS} epochs...")
history = model.fit(
    train_dataset,
    epochs=EPOCHS,
    validation_data=val_dataset
)

print(f"\nModel training completed for {EPOCHS} epochs.")

Mounted at /content/drive
Datasets (train_dataset, val_dataset, test_dataset) have been successfully re-created.

Starting model training for 10 epochs...
Epoch 1/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 34s 107ms/step - accuracy: 0.5436 - loss: 1.8243 - val_accuracy: 0.8791 - val_loss: 0.3854
Epoch 2/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 11s 62ms/step - accuracy: 0.9592 - loss: 0.1711 - val_accuracy: 0.8913 - val_loss: 0.3787
Epoch 3/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 21s 49ms/step - accuracy: 0.9784 - loss: 0.1016 - val_accuracy: 0.8832 - val_loss: 0.3617
Epoch 4/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step - accuracy: 0.9911 - loss: 0.0584 - val_accuracy: 0.9049 - val_loss: 0.3031
Epoch 5/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 12s 64ms/step - accuracy: 0.9967 - loss: 0.0304 - val_accuracy: 0.9008 - val_loss: 0.3175
Epoch 6/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 11s 55ms/step - accuracy: 0.9987 - loss: 0.0205 - val_accuracy: 0.9008 - val_loss: 0.3257
Epoch 7/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 11s 57ms/step - accuracy: 0.9988 - los

In [11]:
import os

# Define a path to save the model weights, correcting the filename to end with .weights.h5
weights_save_path = './fine_tuned_mobilenetv2.weights.h5'

# Save only the model weights
model.save_weights(weights_save_path)

print(f"Model weights saved successfully to {weights_save_path}")

Model weights saved successfully to ./fine_tuned_mobilenetv2.weights.h5


## Đánh giá mô hình

### Subtask:
Đánh giá hiệu suất của mô hình đã tinh chỉnh trên tập xác thực hoặc tập kiểm tra.


**Reasoning**:
As per the instructions, I need to evaluate the trained model on the `test_dataset` using the `.evaluate()` method to get the test loss and test accuracy. The `model` and `test_dataset` are already prepared.



In [9]:
print("\nEvaluating model on the test dataset...")
loss, accuracy = model.evaluate(test_dataset)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


Evaluating model on the test dataset...
115/115 ━━━━━━━━━━━━━━━━━━━━ 23s 202ms/step - accuracy: 0.8741 - loss: 0.4593
Test Loss: 0.3663
Test Accuracy: 0.8970


## Final Task

### Subtask:
Tóm tắt quá trình tinh chỉnh và kết quả đạt được từ mô hình MobileNetV2.


## Summary:

### Data Analysis Key Findings

*   **GPU Availability:** The environment was successfully configured with a GPU (Tesla T4 with 15360MiB memory and CUDA 12.4), enabling accelerated model training.
*   **Data Preparation:** The necessary datasets (`train_dataset`, `val_dataset`, `test_dataset`) were successfully created, following the re-mounting of Google Drive and extraction of the `datasetoxfordclass.zip` file, which contained annotations and images for 37 distinct pet breeds.
*   **Model Architecture:** A MobileNetV2 base model (pre-trained on ImageNet) was used, with its layers frozen, and a new classification head (GlobalAveragePooling2D, Dense(128, relu), Dense(37, softmax)) was added for breed classification.
*   **Training Performance:** The model was trained for 10 epochs using an Adam optimizer and SparseCategoricalCrossentropy loss.
    *   **Initial Performance (Epoch 1):** The model started with a training loss of 1.8243 and accuracy of 0.5436. Validation performance was strong, with a loss of 0.3854 and accuracy of 0.8791.
    *   **Final Training Performance (Epoch 10):** Training loss significantly decreased to 0.0072, and training accuracy reached 1.0000. Validation loss improved to 0.3222, and validation accuracy reached 0.8995.
*   **Test Set Performance:** The fine-tuned model achieved a Test Loss of 0.3663 and a Test Accuracy of 0.8970 on unseen data.

### Insights or Next Steps

*   The transfer learning approach with MobileNetV2 and a custom classification head proved highly effective, achieving nearly 90% accuracy on both validation and test datasets within just 10 epochs.
*   Further improvements could be explored by unfreezing some of the MobileNetV2 base layers and continuing fine-tuning with a lower learning rate, or by implementing data augmentation techniques during training to potentially enhance generalization and prevent overfitting to the training set.
